In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from main import *

from sklearn.cluster import KMeans
from datasetUtils import load_from_Jadson
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# if __name__ == '__main__':
# parser = argparse.ArgumentParser(description='Define the UDA parameters')
#
# parser.add_argument('--gpu_ids', type=str, default="7", help='GPU IDs')
# parser.add_argument('--lr', type=float, default=3.5e-4, help='Learning Rate')
# parser.add_argument('--P', type=int, default=16, help='Number of Persons')
# parser.add_argument('--K', type=int, default=4, help='Number of samples per person')
# parser.add_argument('--tau', type=float, default=0.05, help='tau value used on softmax triplet loss')
# parser.add_argument('--beta', type=float, default=0.999, help='beta used on self-Ensembling')
# parser.add_argument('--k1', type=int, default=30, help='k on k-Reciprocal Encoding')
# parser.add_argument('--sampling', type=str, default="mean", help='Mean or Random feature vectors to be prototype')
# parser.add_argument('--lambda_hard', type=float, default=0.5, help='tuning prameter of Softmax Triplet Loss')
# parser.add_argument('--num_iter', type=int, default=400, help='Number of iterations on an epoch')
# parser.add_argument('--momentum_on_feature_extraction', type=int, default=0,
# help='If it is the momentum used on feature extraction')
# parser.add_argument('--target', type=str, help='Name of target dataset')
# parser.add_argument('--path_to_save_models', type=str, help='Path to save models')
# parser.add_argument('--path_to_save_metrics', type=str, help='Path to save metrics (mAP, CMC, ...)')
# parser.add_argument('--version', type=str, help='Path to save models')
# parser.add_argument('--eval_freq', type=int, help='Evaluation Frequency along training')

# args = parser.parse_args()
# gpu_ids = args.gpu_ids
# base_lr = args.lr
# P = args.P
# K = args.K

# tau = args.tau
# beta = args.beta
# k1 = args.k1
# sampling  = args.sampling
#
# lambda_hard = args.lambda_hard
# number_of_iterations = args.num_iter
# momentum_on_feature_extraction = bool(args.momentum_on_feature_extraction)
# target = args.target
# dir_to_save = args.path_to_save_models
# dir_to_save_metrics = args.path_to_save_metrics
# version = args.version
# eval_freq = args.eval_freq
# main.py --gpu_ids=0,1,2,3 --lr=3.5e-4 --P=16 --K=12 --tau=0.04 --beta=0.999 --k1=30 --sampling=mean --lambda_hard=0.5 --num_iter=7 --momentum_on_feature_extraction=0 --target=Duke --path_to_save_models=models --path_to_save_metrics=metrics --version=version_name --eval_freq=5

import sys
import os
import pandas as pd

from IPython.display import display, Image

from IPython.display import display, HTML
from bs4 import BeautifulSoup


# Função para exibir a imagem usando HTML
def exibir_imagem(imagem_path):
    return f'<img src="{imagem_path}" width="400">'


class Logger:
    def __init__(self, log_file):
        self.terminal = sys.stdout
        self.log_file = log_file

    def write(self, message):
        self.terminal.write(message)
        with open(self.log_file, 'a') as log: # fecha depois de usar
            log.write(message)

from metricas import *

html_content= ""
df = pd.DataFrame({
    'k':[], 
    'lambda_hard':[], 
    'log':[], 
    'matriz_confusao':[], 
    'Acuracia':[], 
    'Precisao':[],
    'Recall':[],
    'F1-score':[],
    'Grafico':[]
    })
#gpus = "2,3,4" # dl-17
gpus = "1,2,3,4"  # dl-17
for k in [8]:
    for lambda_hard in [ 0.2, 0.1, 0.0]:
                
        print(f"**** inicio do teste com ruido em k:{k} e lambda_hard:{lambda_hard} ****")
        
        # iniciar arquivo de log: logJadson_k_lambda_hard.txt
        log = f"logJadson_{k}_{lambda_hard}.txt"
        if(os.path.exists(log)):
            os.remove(log)
                
        # Redirecionar a saída padrão para o arquivo de log
        # logger = Logger(log)
        # sys.stdout = logger
        
        version = f"teste-02-ruido_{k}_{lambda_hard}"
        main(gpu_ids=gpus,base_lr=3.5e-4,P=16,K=k,tau=0.04,beta=0.999,k1=30,sampling="random",lambda_hard=lambda_hard,number_of_iterations=7,momentum_on_feature_extraction=0,target="Jadson",dir_to_save="models",dir_to_save_metrics="metrics",version=version,eval_freq=5,use_ruido=True)
        
        # Restaurar a saída padrão para a tela
        # sys.stdout = logger.terminal
        
        metricas_t, metricas_v = metricas(k=k,lambda_hard=lambda_hard)
        
        for count in range( 0, metricas_t.shape[0] ):
            linha = {
            'k':[k], 
            'lambda_hard':[lambda_hard], 
            'log':[log], 
            'matriz_confusao':[f'resultados/MC_{k}_{lambda_hard}_{count}_test.png'], 
            'Acuracia': [metricas_t[count][0]], 
            'Precisao': [metricas_t[count][1]], 
            'Recall':   [metricas_t[count][2]],
            'F1-score': [metricas_t[count][3]], 
            'Grafico':  [f'resultados/grafico_{k}_{lambda_hard}_{count}_test.png'],
            'Tipo':     'Test'
                }
            df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
            
        for count in range( 0, metricas_v.shape[0] ):
            linha = {
            'k':[k], 
            'lambda_hard':[lambda_hard], 
            'log':[log], 
            'matriz_confusao':[f'resultados/MC_{k}_{lambda_hard}_{count}_valid.png'], 
            'Acuracia': [metricas_v[count][0]], 
            'Precisao': [metricas_v[count][1]], 
            'Recall':   [metricas_v[count][2]], 
            'F1-score': [metricas_v[count][3]], 
            'Grafico':  [f'resultados/grafico_{k}_{lambda_hard}_{count}_valid.png'],
            'Tipo':     'Valid'
                }
            df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
        
        # Aplicar a função à coluna 'imagem' e criar uma nova coluna 'imagem_exibicao'
        df['MC'] = df['matriz_confusao'].apply(exibir_imagem)
        df['GR'] = df['Grafico'].apply(exibir_imagem)
        
        html_content = df[['k', 'lambda_hard', 'Acuracia', 'Precisao', 'Recall', 'F1-score', 'Tipo', 'MC', 'GR']].to_html(escape=False, index=False)
        # salvando df em arquivo html
        # Use BeautifulSoup para formatar o HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        formatted_html = soup.prettify()
        
        # Salve o HTML em um arquivo
        with open('relatorio-17-2.html', 'w', encoding='utf-8') as file:
            file.write('<html><head><title>Meu Relatório</title></head><body>')
            file.write(formatted_html)
            file.write('</body></html>')



/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly

**** inicio do teste com ruido em k:8 e lambda_hard:0.2 ****
Num GPU's: 4
Allocated GPU's for model: [1, 2, 3]


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 72.07978320121765
Computing jaccard distance...


bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 88.03822088241577
Reliability: 0.996
Mean Purity: 0.03106
There are 2 clusters with 5 cameras
There are 3 clusters with 7 cameras
There are 2 clusters with 8 cameras
There are 1 clusters with 9 cameras
There are 1 clusters with 10 cameras
There are 2 clusters with 11 cameras
There are 1 clusters with 12 cameras
There are 1 clusters with 15 cameras
There are 1 clusters with 19 cameras
There are 1 clusters with 21 cameras
There are 1 clusters with 24 cameras
There are 1 clusters with 28 cameras
There are 1 clusters with 36 cameras
There are 1 clusters with 40 cameras
There are 1 clusters with 45 cameras
There are 1 clusters with 50 cameras
There are 1 clusters with 51 cameras
There are 1 clusters with 53 cameras
There are 1 clusters with 55 cameras
There are 2 clusters with 56 cameras
There are 2 clusters with 57 cameras
There are 1 clusters with 59 cameras
There are 2 clusters with 60 cameras
There are 12 clusters with 61 cameras
There are 11 cluste

/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/emorais/miniconda3/envs/myenv/lib/python3.10/s

In [2]:
# Exibir o DataFrame com as imagens
display(HTML(html_content))
print(df)

k,lambda_hard,Acuracia,Precisao,Recall,F1-score,Tipo,MC,GR
8.0,0.2,0.348833,0.675029,0.358750,0.468508,Test,,
8.0,0.2,0.348833,0.675029,0.358750,0.468508,Test,,
8.0,0.2,0.651083,0.892417,0.641146,0.746196,Test,,
8.0,0.2,0.348792,0.675032,0.358646,0.468419,Test,,
8.0,0.2,0.348833,0.675029,0.358750,0.468508,Test,,
8.0,0.2,0.651167,0.892376,0.641302,0.746288,Test,,
8.0,0.2,0.349167,0.675284,0.359167,0.468924,Test,,
8.0,0.2,0.651125,0.892368,0.641250,0.746250,Test,,
8.0,0.2,0.348833,0.675064,0.358698,0.468472,Test,,
8.0,0.2,0.348833,0.675029,0.358750,0.468508,Test,,


     k  lambda_hard                  log                  matriz_confusao  \
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_0_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_1_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_2_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_3_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_4_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_5_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_6_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_7_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_8_test.png   
0  8.0          0.2  logJadson_8_0.2.txt   resultados/MC_8_0.2_9_test.png   
0  8.0          0.2  logJadson_8_0.2.txt  resultados/MC_8_0.2_0_valid.png   
0  8.0          0.2  logJadson_8_0.2.txt  resultados/MC_8_0.2_1_valid.png   